In [ ]:
import mitsuba as mi
mi.set_variant('cuda_ad_rgb')
from mitsuba import ScalarTransform4f as ST
from pathlib import Path

## Mesh reading, image taking

In [ ]:
obj = 'Weisshai_Great_White_Shark'
obj_path = Path(f'../data/raw_objects/{obj}/').absolute()

mesh: mi.Mesh = mi.load_dict({
    'type': 'obj',
    'filename': (obj_path / 'meshes/model.obj').as_posix(),
})

bbox = mesh.bbox()  # Used for re-centering and scaling to -1:1 bounding box

mesh: mi.Mesh = mi.load_dict({
    'type': 'obj',
    'filename': (obj_path / 'meshes/model.obj').as_posix(),
    'to_world': ST().scale(1 / max(abs(bbox.max - bbox.min) / 2)).translate(-(bbox.max + bbox.min) / 2),
    'bsdf': {
        'type': 'diffuse',
        'reflectance': {
            'type': 'bitmap',
            'filename': (obj_path / 'materials/textures/texture.png').as_posix(),
            'wrap_mode': 'clamp',
            'filter_type': 'bilinear',
        }
    }
})

mesh.bbox()

In [ ]:
scene: mi.Scene = mi.load_dict({
    'type': 'scene',
    # The keys below correspond to object IDs and can be chosen arbitrarily
    'integrator': {'type': 'path'},
    'light': {
        'type': 'constant',
        'radiance': {
            'type': 'rgb',
            'value': 1.0
        }
    },
    'obj': mesh
})

def load_sensor(r, phi, theta) -> list[mi.Sensor]:
    # Apply two rotations to convert from spherical coordinates to world 3D coordinates.
    origin = ST().rotate([0, 0, 1], phi).rotate([0, 1, 0], theta) @ mi.ScalarPoint3f([0, 0, r])

    return mi.load_dict({
        'type': 'perspective',
        'fov': 40,
        'fov_axis': 'x',
        'to_world': ST().look_at(
            origin=origin,
            target=[0, 0, 0],
            up=[0, 0, 1]
        ),
        'sampler': {
            'type': 'ldsampler',
            'sample_count': 64,
        },
        'film': {
            'type': 'hdrfilm',
            'width': 800,
            'height': 800,
            'pixel_format': 'rgba',
            'filter': {
                'type': 'gaussian'
            }
        },
    })

sensor_count = 6

radius = 4
phis = [60.0 * i for i in range(sensor_count)]
theta = 90.0

sensors: list[mi.Sensor] = [load_sensor(radius, phi, theta) for phi in phis]
images = [mi.render(scene, sensor=sensor) for sensor in sensors]

from matplotlib import pyplot as plt
fig = plt.figure(figsize=(20, 14))
fig.subplots_adjust(wspace=0, hspace=0)
for i in range(sensor_count):
    ax = fig.add_subplot(2, 3, i + 1).imshow(images[i] ** (1.0 / 2.2))
    plt.axis("off")

mi.util.write_bitmap("render.png", images[5])